In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torchsummary import summary
    
import os
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim

In [2]:
! mkdir saved_image

mkdir: cannot create directory ‘saved_image’: File exists


In [4]:
import segmentation_models_pytorch as smp

def get_pretrained_unetplusplus(input_channel, output_channel):
    model = smp.UnetPlusPlus(
        encoder_name="resnet34", # Choose the encoder architecture
        encoder_weights="imagenet", # Use pre-trained weights from ImageNet
        in_channels=input_channel,
        classes=output_channel,
        activation=None, # No activation function for the final layer
    )
    return model


In [5]:

import os
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

class LitsDataset(Dataset):
    def __init__(self, csv_file, base_path, type="train", split_ratio=0.2, transform=None, extra_samples=1000):
        self.df = pd.read_csv(csv_file)
        self.extra = pd.read_csv(csv_file)
        self.base_path = base_path
        self.transform = transform
        
        # Filter out liver_mask_empty rows
        self.df = self.df[self.df['liver_mask_empty'].isin(['TRUE', True])]
        # Add 3000 extra images where liver_mask_empty is False
        extra_df = self.extra[self.extra['liver_mask_empty'].isin(['False', False])]
        extra_df1 = extra_df.head(extra_samples)
        extra_df2 = extra_df.tail(extra_samples)
        self.df1 = pd.concat([extra_df1 , self.df], ignore_index=True)
        self.df = pd.concat([self.df1, extra_df2], ignore_index=True)

        # Train-test split
        if type=="train":
            self.df = self.df.iloc[:int(len(self.df)*(1-split_ratio))]
        elif type=="val":
            self.df = self.df.iloc[int(len(self.df)*(1-split_ratio)):]

        self.df['tumor_present'] = self.df['tumor_mask_empty'].apply(lambda x: x not in ['TRUE', True])
        

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = os.path.join(self.base_path, row['filepath'])
        image = np.array(Image.open(img_path).convert("RGB"), dtype=np.float32)  # Cast to float32
        image = np.transpose(image, (2, 0, 1))

        # Load tumor and liver masks
        tumor_mask_path = os.path.join(self.base_path, row['tumor_maskpath'])
        liver_mask_path = os.path.join(self.base_path, row['liver_maskpath'])
        tumor_mask = np.array(Image.open(tumor_mask_path).point(lambda x: x * 255).convert("L"), dtype=np.float32)
        liver_mask = np.array(Image.open(liver_mask_path).point(lambda x: x * 255).convert("L"), dtype=np.float32)

        # Set mask values
        tumor_mask[tumor_mask == 255.0] = 1.0
        liver_mask[liver_mask == 255.0] = 1.0

        # Create a two-channel mask
        mask = np.stack((liver_mask, tumor_mask), axis=0)

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        has_tumor = self.df.iloc[index]['tumor_present']

        return image, mask, has_tumor


In [6]:
import torchvision
from torch.utils.data import DataLoader
import torch


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


In [ ]:
from torch.utils.data import DataLoader
def get_loaders(
    csv_file,
    base_path,
    batch_size=8,
    train_transform=None,
    val_transform=None,
    num_workers=4,
    pin_memory=True,
):

    train_ds = LitsDataset(
        csv_file=csv_file,
        base_path=base_path,
        type="train",
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = LitsDataset(
        csv_file=csv_file,
        base_path=base_path,
        type="val",
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    return train_loader, val_loader




In [7]:
def check_val(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score_1 = 0
    dice_score_2 = 0
    model.eval()

    with torch.no_grad():
        for x, y, has_tumor in loader:  
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds_1 = preds[:, 0, :, :]
            preds_2 = preds[:, 1, :, :]
            y_1 = y[:, 0, :, :]
            y_2 = y[:, 1, :, :]
            preds_1 = (preds_1 > 0.5).float()
            preds_2 = (preds_2 > 0.5).float()
            num_correct += ((preds_1 == y_1) & (preds_2 == y_2)).sum()
            num_pixels += torch.numel(y_1)
            dice_score_1 += (2 * (preds_1 * y_1).sum()) / ((preds_1 + y_1).sum() + 1e-8)
            dice_score_2 += (2 * (preds_2 * y_2).sum()) / ((preds_2 + y_2).sum() + 1e-8)

    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Channel 1 Dice score: {dice_score_1/len(loader)}")
    print(f"Channel 2 Dice score: {dice_score_2/len(loader)}")
    print(f"Average Dice score: {(dice_score_1 + dice_score_2) / (2 * len(loader))}")
    model.train()


In [8]:
import numpy as np
from scipy.spatial.distance import directed_hausdorff
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score_1 = 0
    dice_score_2 = 0
    jaccard_score_1 = 0
    jaccard_score_2 = 0
    precision_1 = 0
    precision_2 = 0
    recall_1 = 0
    recall_2 = 0
    model.eval()
    hausdorff_distance_1 = 0
    hausdorff_distance_2 = 0
    with torch.no_grad():
        for x, y , has_tumor in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds_1 = preds[:, 0, :, :]
            preds_2 = preds[:, 1, :, :]
            y_1 = y[:, 0, :, :]
            y_2 = y[:, 1, :, :]
            preds_1 = (preds_1 > 0.5).float()
            preds_2 = (preds_2 > 0.5).float()

            intersection_1 = preds_1 * y_1
            intersection_2 = preds_2 * y_2

            num_correct += (intersection_1.sum() + intersection_2.sum())
            num_pixels += (y_1.sum() + y_2.sum())

            dice_score_1 += (2 * intersection_1.sum()) / (preds_1.sum() + y_1.sum() + 1e-8)
            dice_score_2 += (2 * intersection_2.sum()) / (preds_2.sum() + y_2.sum() + 1e-8)

            union_1 = preds_1 + y_1 - intersection_1
            union_2 = preds_2 + y_2 - intersection_2
            jaccard_score_1 += intersection_1.sum() / (union_1.sum() + 1e-8)
            jaccard_score_2 += intersection_2.sum() / (union_2.sum() + 1e-8)

            precision_1 += intersection_1.sum() / (preds_1.sum() + 1e-8)
            precision_2 += intersection_2.sum() / (preds_2.sum() + 1e-8)
            recall_1 += intersection_1.sum() / (y_1.sum() + 1e-8)
            recall_2 += intersection_2.sum() / (y_2.sum() + 1e-8)
            for i in range(preds_1.shape[0]):
                y_np_1 = y_1[i].cpu().numpy()
                preds_np_1 = preds_1[i].cpu().numpy()
                y_np_2 = y_2[i].cpu().numpy()
                preds_np_2 = preds_2[i].cpu().numpy()
                
                hd1 = directed_hausdorff(y_np_1, preds_np_1)[0]
                hd2 = directed_hausdorff(y_np_2, preds_np_2)[0]
                
                hausdorff_distance_1 += hd1
                hausdorff_distance_2 += hd2

    n = len(loader)
    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Channel 1 Dice score: {dice_score_1/n}")
    print(f"Channel 2 Dice score: {dice_score_2/n}")
    print(f"Average Dice score: {(dice_score_1 + dice_score_2) / (2 * n)}")
    print(f"Channel 1 Jaccard score: {jaccard_score_1/n}")
    print(f"Channel 2 Jaccard score: {jaccard_score_2/n}")
    print(f"Average Jaccard score: {(jaccard_score_1 + jaccard_score_2) / (2 * n)}")
    print(f"Channel 1 Precision: {precision_1/n}")
    print(f"Channel 2 Precision: {precision_2/n}")
    print(f"Average Precision: {(precision_1 + precision_2) / (2 * n)}")
    print(f"Channel 1 Recall: {recall_1/n}")
    print(f"Channel 2 Recall: {recall_2/n}")
    print(f"Average Recall: {(recall_1 + recall_2) / (2 * n)}")
    print(f"Channel 1 F1-score: {2 * (precision_1 * recall_1) / (precision_1 + recall_1 + 1e-8)/n}")
    print(f"Channel 2 F1-score: {2 * (precision_2 * recall_2) / (precision_2 + recall_2 + 1e-8)/n}")
    print(f"Average F1-score: {(2 * (precision_1 * recall_1) + 2 * (precision_2 * recall_2)) / (precision_1 + recall_1 + precision_2 + recall_2 + 1e-8)/n}")
    n = len(loader) * x.shape[0]
    print(f"Channel 1 Hausdorff distance: {hausdorff_distance_1/n}")
    print(f"Channel 2 Hausdorff distance: {hausdorff_distance_2/n}")
    print(f"Average Hausdorff distance: {(hausdorff_distance_1 + hausdorff_distance_2) / (2 * n)}")

In [9]:
def save_predictions_as_imgs(loader, model, folder="saved_image/", device="cuda"):
    model.eval()
    for idx, (x, y, has_tumor) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        # Save liver masks and predictions
        torchvision.utils.save_image(y[:, 0:1, :, :], f"{folder}/mask_liver_{idx}.png")
        torchvision.utils.save_image(preds[:, 0:1, :, :], f"{folder}/pred_liver_{idx}.png")

        # Save tumor masks and predictions only if has_tumor is True
        if has_tumor.any():
            torchvision.utils.save_image(y[:, 1:2, :, :], f"{folder}/mask_tumor_{idx}.png")
            torchvision.utils.save_image(preds[:, 1:2, :, :], f"{folder}/pred_tumor_{idx}.png")

    model.train()


In [10]:
# hyperparams
lr = 1e-6
dev = "cuda"
batch_size = 16
epochs = 10
workers= 8
img_h = 256
img_w = 256
pin_mem= True
load_model = False


In [11]:
from tqdm import tqdm

def train_fn(loader, model, optimizer, loss_fn, scaler, device):
    loop = tqdm(loader)

    for batch_idx, (data, targets, has_tumor) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.float().to(device=device)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            
            # Calculate loss for liver masks (always)
            liver_targets = targets[:, 0, :, :]
            liver_outputs = predictions[:, 0, :, :]
            liver_loss = loss_fn(liver_outputs, liver_targets)

            # Calculate loss for tumor masks (only when has_tumor is True)
            if has_tumor.any():
                tumor_targets = targets[:, 1, :, :]
                tumor_outputs = predictions[:, 1, :, :]
                tumor_loss = loss_fn(tumor_outputs, tumor_targets)
                loss = liver_loss + tumor_loss
            else:
                loss = liver_loss

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


In [12]:
def eval_fn(loader, model, device, criterion):
    model.eval()
    liver_total_loss = 0.0
    tumor_total_loss = 0.0
    num_samples = 0
    num_samples_with_tumor = 0
    
    with torch.no_grad():
        for data in loader:
            inputs, targets, has_tumor = data
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            
            # Calculate loss for liver masks (always)
            liver_targets = targets[:, 0, :, :]
            liver_outputs = outputs[:, 0, :, :]
            liver_loss = criterion(liver_outputs, liver_targets)
            liver_total_loss += liver_loss.item() * inputs.size(0)
            
            # Calculate loss for tumor masks (only when any item in the batch has a tumor)
            if has_tumor.any():
                tumor_targets = targets[:, 1, :, :]
                tumor_outputs = outputs[:, 1, :, :]
                tumor_loss = criterion(tumor_outputs, tumor_targets)
                tumor_total_loss += tumor_loss.item() * inputs.size(0)
                num_samples_with_tumor += inputs.size(0)
            num_samples += inputs.size(0)
        
        avg_liver_loss = liver_total_loss / num_samples
        avg_tumor_loss = tumor_total_loss / num_samples_with_tumor if num_samples_with_tumor > 0 else 0
    
    return avg_liver_loss, avg_tumor_loss


In [13]:

import torch
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss
import torch.nn as nn

class WeightedDiceLoss(nn.Module):
    def __init__(self, weights, mode='multilabel', eps=1e-7):
        super(WeightedDiceLoss, self).__init__()
        self.weights = weights
        self.mode = mode
        self.eps = eps

    def forward(self, input, target):
        assert input.size() == target.size(), "Input and target must have the same size"
        input = torch.sigmoid(input)
        
        if self.mode == 'multilabel':
            weight_map = torch.zeros_like(target)
            for i, w in enumerate(self.weights):
                weight_map[target == i] = w
            intersection = torch.sum(weight_map * input * target)
            union = torch.sum(weight_map * (input + target))
        else:
            intersection = torch.sum(input * target)
            union = torch.sum(input + target)

        dice = (2 * intersection + self.eps) / (union + self.eps)

        return 1 - dice
weights = torch.tensor([0.5, 5]).to(dev)  # Give more weight to white pixels (class 1) and less to black pixels (class 0)
loss_fn = WeightedDiceLoss(weights)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
def get_transforms(resize_height, resize_width):
    train_transform = A.Compose([
        A.Resize(resize_height, resize_width),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        #A.Normalize(),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(resize_height, resize_width),
        #A.Normalize(),
        ToTensorV2(),
    ])

    return train_transform, val_transform
train_transform, val_transform = get_transforms(resize_height=256, resize_width=256)

In [15]:

input_channel = 3
output_channel = 2  # Updated from 1 to 2
model = get_pretrained_unetplusplus(input_channel, output_channel).to(dev)

from segmentation_models_pytorch.losses import DiceLoss
#loss_fn = DiceLoss(mode='multilabel')


optimizer = optim.AdamW(model.parameters(), lr=5e-4 ,  weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)



train_loader, val_loader = get_loaders(
    csv_file="/apps/local/shared/HC701/assessment/assignment_3/data/hc701_lits_train.csv",
    base_path="/apps/local/shared/HC701/assessment/assignment_3/data/",
    batch_size=64,
    train_transform=None,
    val_transform=None,
)

#check_accuracy(val_loader, model, device=dev)
scaler = torch.cuda.amp.GradScaler()

# test_accuracy(test_loader, model, device=dev)
# check_accuracy(val_loader, model, device=dev)

for epoch in range(epochs):
    print(epoch)
    train_fn(train_loader, model, optimizer, loss_fn, scaler, device=dev)
    val_loss = eval_fn(val_loader, model, device=dev, criterion=loss_fn)
    avg_liver_loss, avg_tumor_loss = eval_fn(val_loader, model, device=dev, criterion=loss_fn)
    avg_loss = avg_liver_loss*0.1 + avg_tumor_loss*0.9
    scheduler.step(avg_loss)

    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict(),
    }
    # check_accuracy(test_loader, model, device=dev)
    # check_accuracy(val_loader, model, device=dev)
    check_val(val_loader, model, device=dev)

    save_predictions_as_imgs(val_loader, model, folder="saved_image/", device=dev)


0


  0%|          | 0/177 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7a886a9c60>
Traceback (most recent call last):
  File "/home/ahmed.sharshar/.conda/envs/new/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/ahmed.sharshar/.conda/envs/new/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1412, in _shutdown_workers
    self._pin_memory_thread.join()
  File "/home/ahmed.sharshar/.conda/envs/new/lib/python3.10/threading.py", line 1096, in join
    self._wait_for_tstate_lock()
  File "/home/ahmed.sharshar/.conda/envs/new/lib/python3.10/threading.py", line 1116, in _wait_for_tstate_lock
    if lock.acquire(block, timeout):
KeyboardInterrupt: 
  0%|          | 0/177 [03:35<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[64, 256, 256, 256] to have 3 channels, but got 256 channels instead

# Test

In [ ]:
class LitsTestDataset(Dataset):
    def __init__(self, csv_file, base_path, transform=None):
        self.df = pd.read_csv(csv_file)
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = os.path.join(self.base_path, row['filepath'])
        image = np.array(Image.open(img_path).convert("RGB"), dtype=np.float32)  # Cast to float32
        image = np.transpose(image, (2, 0, 1))

        # Load tumor and liver masks
        tumor_mask_path = os.path.join(self.base_path, row['tumor_maskpath'])
        liver_mask_path = os.path.join(self.base_path, row['liver_maskpath'])
        tumor_mask = np.array(Image.open(tumor_mask_path).point(lambda x: x * 255).convert("L"), dtype=np.float32)
        liver_mask = np.array(Image.open(liver_mask_path).point(lambda x: x * 255).convert("L"), dtype=np.float32)

        # Set mask values
        tumor_mask[tumor_mask == 255.0] = 1.0
        liver_mask[liver_mask == 255.0] = 1.0

        # Create a two-channel mask
        mask = np.stack((liver_mask, tumor_mask), axis=0)

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask


In [ ]:
def test_loaders(
    csv_file,
    base_path,
    batch_size=4,
    test_transform=None,
    num_workers=4,
    pin_memory=True
):
    test_ds = LitsTestDataset(
    csv_file=csv_file,
    base_path=base_path,
    transform=test_transform,
    )

    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return test_loader

In [ ]:
test_loader = test_loaders(
    csv_file="/apps/local/shared/HC701/assessment/assignment_3/data/hc701_lits_test.csv",
    base_path="/apps/local/shared/HC701/assessment/assignment_3/data/",
    batch_size=16,
    test_transform=None,
)

In [ ]:
import numpy as np
from scipy.spatial.distance import directed_hausdorff
def test_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score_1 = 0
    dice_score_2 = 0
    jaccard_score_1 = 0
    jaccard_score_2 = 0
    precision_1 = 0
    precision_2 = 0
    recall_1 = 0
    recall_2 = 0
    model.eval()
    hausdorff_distance_1 = 0
    hausdorff_distance_2 = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds_1 = preds[:, 0, :, :]
            preds_2 = preds[:, 1, :, :]
            y_1 = y[:, 0, :, :]
            y_2 = y[:, 1, :, :]
            preds_1 = (preds_1 > 0.5).float()
            preds_2 = (preds_2 > 0.5).float()

            intersection_1 = preds_1 * y_1
            intersection_2 = preds_2 * y_2

            num_correct += (intersection_1.sum() + intersection_2.sum())
            num_pixels += (y_1.sum() + y_2.sum())

            dice_score_1 += (2 * intersection_1.sum()) / (preds_1.sum() + y_1.sum() + 1e-8)
            dice_score_2 += (2 * intersection_2.sum()) / (preds_2.sum() + y_2.sum() + 1e-8)

            union_1 = preds_1 + y_1 - intersection_1
            union_2 = preds_2 + y_2 - intersection_2
            jaccard_score_1 += intersection_1.sum() / (union_1.sum() + 1e-8)
            jaccard_score_2 += intersection_2.sum() / (union_2.sum() + 1e-8)

            precision_1 += intersection_1.sum() / (preds_1.sum() + 1e-8)
            precision_2 += intersection_2.sum() / (preds_2.sum() + 1e-8)
            recall_1 += intersection_1.sum() / (y_1.sum() + 1e-8)
            recall_2 += intersection_2.sum() / (y_2.sum() + 1e-8)
            for i in range(preds_1.shape[0]):
                y_np_1 = y_1[i].cpu().numpy()
                preds_np_1 = preds_1[i].cpu().numpy()
                y_np_2 = y_2[i].cpu().numpy()
                preds_np_2 = preds_2[i].cpu().numpy()
                
                hd1 = directed_hausdorff(y_np_1, preds_np_1)[0]
                hd2 = directed_hausdorff(y_np_2, preds_np_2)[0]
                
                hausdorff_distance_1 += hd1
                hausdorff_distance_2 += hd2

    n = len(loader)
    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Channel 1 Dice score: {dice_score_1/n}")
    print(f"Channel 2 Dice score: {dice_score_2/n}")
    print(f"Average Dice score: {(dice_score_1 + dice_score_2) / (2 * n)}")
    print(f"Channel 1 Jaccard score: {jaccard_score_1/n}")
    print(f"Channel 2 Jaccard score: {jaccard_score_2/n}")
    print(f"Average Jaccard score: {(jaccard_score_1 + jaccard_score_2) / (2 * n)}")
    print(f"Channel 1 Precision: {precision_1/n}")
    print(f"Channel 2 Precision: {precision_2/n}")
    print(f"Average Precision: {(precision_1 + precision_2) / (2 * n)}")
    print(f"Channel 1 Recall: {recall_1/n}")
    print(f"Channel 2 Recall: {recall_2/n}")
    print(f"Average Recall: {(recall_1 + recall_2) / (2 * n)}")
    print(f"Channel 1 F1-score: {2 * (precision_1 * recall_1) / (precision_1 + recall_1 + 1e-8)/n}")
    print(f"Channel 2 F1-score: {2 * (precision_2 * recall_2) / (precision_2 + recall_2 + 1e-8)/n}")
    print(f"Average F1-score: {(2 * (precision_1 * recall_1) + 2 * (precision_2 * recall_2)) / (precision_1 + recall_1 + precision_2 + recall_2 + 1e-8)/n}")
    n = len(loader) * x.shape[0]
    print(f"Channel 1 Hausdorff distance: {hausdorff_distance_1/n}")
    print(f"Channel 2 Hausdorff distance: {hausdorff_distance_2/n}")
    print(f"Average Hausdorff distance: {(hausdorff_distance_1 + hausdorff_distance_2) / (2 * n)}")

In [ ]:
test_accuracy(test_loader, model, device=dev)

Got 6738262.0/7501262.0 with acc 89.83
Channel 1 Dice score: 0.5945385694503784
Channel 2 Dice score: 0.1304144561290741
Average Dice score: 0.36247649788856506
Channel 1 Jaccard score: 0.5519060492515564
Channel 2 Jaccard score: 0.09957405924797058
Average Jaccard score: 0.3257400691509247
Channel 1 Precision: 0.5725361108779907
Channel 2 Precision: 0.2362295538187027
Average Precision: 0.4043827950954437
Channel 1 Recall: 0.6336069703102112
Channel 2 Recall: 0.1109042838215828
Average Recall: 0.3722556233406067
Channel 1 F1-score: 0.6015254259109497
Channel 2 F1-score: 0.15094390511512756
Average F1-score: 0.5008273124694824
Channel 1 Hausdorff distance: 9.626958103826295
Channel 2 Hausdorff distance: 7.44515377091632
Average Hausdorff distance: 8.536055937371307
